# Pickling cuML Models for Persistence

This notebook demonstrates simple pickling of both single-GPU and multi-GPU cuML models for persistence

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Single GPU Model Pickling

All single-GPU estimators are pickleable. The following example demonstrates the creation of a synthetic dataset, training, and pickling of the resulting model for storage. Trained single-GPU models can also be used to distribute the inference on a Dask cluster, which the `Distributed Model Pickling` section below demonstrates.

In [2]:
from cuml.datasets import make_blobs

X, y = make_blobs(n_samples=50,
                  n_features=10,
                  centers=5,
                  cluster_std=0.4,
                  random_state=0)

In [3]:
from cuml.cluster import KMeans

model = KMeans(n_clusters=5)

model.fit(X)

KMeans()

In [4]:
import pickle

pickle.dump(model, open("kmeans_model.pkl", "wb"))

In [5]:
model = pickle.load(open("kmeans_model.pkl", "rb"))

In [6]:
model.cluster_centers_

array([[-5.768463  ,  2.3276033 , -3.7457771 , -1.8541754 , -5.1695833 ,
         7.667088  ,  2.7118318 ,  8.495609  ,  1.7038484 ,  1.1884269 ],
       [ 4.6476874 ,  8.377879  , -9.070581  ,  9.459332  ,  8.450422  ,
        -1.0210547 ,  3.3920872 , -7.8629856 , -0.75276613,  0.4838412 ],
       [-2.9414442 ,  4.6401706 , -4.502753  ,  2.2855108 ,  1.644645  ,
        -2.4937892 , -5.2241607 , -1.5499198 , -8.063638  ,  2.816936  ],
       [-4.271077  ,  5.561165  , -5.6640916 , -1.8229512 , -9.2925    ,
         0.73028314,  4.4586773 , -2.8876226 , -5.1257744 ,  9.694357  ],
       [ 5.5837417 , -4.1515303 ,  4.369667  , -3.00205   ,  3.6388965 ,
        -4.341912  , -3.3187115 ,  6.503671  , -6.865036  , -1.0266498 ]],
      dtype=float32)

## Distributed Model Pickling

The distributed estimator wrappers inside of the `cuml.dask` are not intended to be pickled directly. The Dask cuML estimators provide a function `get_combined_model()`, which returns the trained single-GPU model for pickling. The combined model can be used for inference on a single-GPU, and the `ParallelPostFit` wrapper from the [Dask-ML](https://ml.dask.org/meta-estimators.html) library can be used to perform distributed inference on a Dask cluster.

In [7]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:38856 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 251.80 GiB


In [8]:
from cuml.dask.datasets import make_blobs

n_workers = len(client.scheduler_info()["workers"].keys())

X, y = make_blobs(n_samples=5000, 
                  n_features=30,
                  centers=5, 
                  cluster_std=0.4, 
                  random_state=0,
                  n_parts=n_workers*5)

X = X.persist()
y = y.persist()

In [9]:
from cuml.dask.cluster import KMeans

dist_model = KMeans(n_clusters=5)

In [10]:
dist_model.fit(X)

In [11]:
import pickle

single_gpu_model = dist_model.get_combined_model()
pickle.dump(single_gpu_model, open("kmeans_model.pkl", "wb"))

In [12]:
single_gpu_model = pickle.load(open("kmeans_model.pkl", "rb"))

In [13]:
single_gpu_model.cluster_centers_

array([[ 4.8098736 ,  8.422671  , -9.239024  ,  9.379143  ,  8.49988   ,
        -1.0592817 ,  3.3437848 , -7.802611  , -0.59463316,  0.26447606,
         5.507395  , -4.1069803 ,  4.2890778 , -2.8172054 ,  3.6150155 ,
        -4.1612997 , -3.6209636 ,  6.2185297 , -6.9460444 , -1.0828307 ,
        -5.8267694 ,  2.2258768 , -3.8601208 , -1.6974077 , -5.313418  ,
         7.5795755 ,  2.9187477 ,  8.540424  ,  1.55232   ,  1.0841808 ],
       [-2.8941844 ,  4.474191  , -4.447568  ,  2.3820987 ,  1.7478832 ,
        -2.5046246 , -5.2083306 , -1.6937684 , -8.134755  ,  2.6468296 ,
        -4.3163633 ,  5.5655403 , -5.732199  , -1.7384955 , -9.344654  ,
         0.70846575,  4.4358406 , -2.9008992 , -4.948638  ,  9.695305  ,
         8.36652   , -6.2474546 , -6.3494725 ,  1.9546976 ,  4.157617  ,
        -9.167904  ,  4.607067  ,  8.788586  ,  6.8644214 ,  2.2319875 ],
       [-4.6657143 , -9.558959  ,  6.657229  ,  4.440131  ,  2.1730301 ,
         2.5904036 ,  0.5800098 ,  6.2550364 , -8